<a href="https://colab.research.google.com/github/Baaabaei/Baaabaei/blob/main/Generators_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 31.4 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import pulp

In [4]:

# Define the generator types
generator_types = [
    {"initial_cost": 2000, "cost_per_mw": 2, "cost_per_hour_at_minimum": 1000, "maximum_level": 2000, "minimum_level": 850},
    {"initial_cost": 1000, "cost_per_mw": 1.3, "cost_per_hour_at_minimum": 2600, "maximum_level": 1750, "minimum_level": 1250},
    {"initial_cost": 500, "cost_per_mw": 3, "cost_per_hour_at_minimum": 3000, "maximum_level": 4000, "minimum_level": 1500},
]

In [5]:
# Define the load requirements
load_requirements = {
    "12 pm to 6 am": 15000,
    "6 to 9 am": 30000,
    "9 am to 3 pm": 25000,
    "3 pm to 6 pm": 40000,
    "6 pm to 12 pm": 27000,
}


In [6]:
# Create a model
model = pulp.LpProblem("Generator Scheduling", pulp.LpMinimize)


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [12]:
# Define generator types as a list or tuple
generator_types = ['type1', 'type2', 'type3']

# Create decision variables for the generator output
generator_output = pulp.LpVariable.dicts("Generator Output", (generator_types, list(load_requirements.keys())), cat=pulp.LpContinuous, lowBound=0)


In [13]:
# Create decision variables for the generator status
generator_status = pulp.LpVariable.dicts("Generator Status", (generator_types, load_requirements.keys()), cat=pulp.LpBinary)

In [16]:
# Define generator types as a list of dictionaries
generator_types = [
    {"initial_cost": 10, "cost_per_hour_at_minimum": 2, "cost_per_mw": 5, "minimum_level": 20},
    {"initial_cost": 15, "cost_per_hour_at_minimum": 3, "cost_per_mw": 7, "minimum_level": 30},
    # Add more generator types as needed
]

# Create decision variables for generator status and output
generator_status = pulp.LpVariable.dicts("Generator Status", [(i, t) for i in range(len(generator_types)) for t in load_requirements.keys()], cat=pulp.LpBinary)
generator_output = pulp.LpVariable.dicts("Generator Output", [(i, t) for i in range(len(generator_types)) for t in load_requirements.keys()], cat=pulp.LpContinuous, lowBound=0)

# Objective function: minimize the total cost
objective = pulp.lpSum([
    generator_types[i]["initial_cost"] * generator_status[(i, t)]
    + generator_types[i]["cost_per_hour_at_minimum"] * generator_status[(i, t)]
    + generator_types[i]["cost_per_mw"] * (generator_output[(i, t)] - generator_types[i]["minimum_level"])
    for i in range(len(generator_types))
    for t in load_requirements.keys()
])
model.setObjective(objective)


In [17]:
# Constraints: meet the load requirements
for t in load_requirements.keys():
    model.addConstraint(pulp.lpSum([generator_output[i, t] for i in range(len(generator_types))]) >= load_requirements[t] * 1.15)


In [19]:
# Constraints: generator output limits
for i in range(len(generator_types)):
    for t in load_requirements.keys():
        # Check if "maximum_level" key exists in generator_types[i] dictionary
        if "maximum_level" in generator_types[i]:
            model.addConstraint(generator_output[(i, t)] <= generator_types[i]["maximum_level"] * generator_status[(i, t)])
        else:
            # Handle the case when "maximum_level" key is not present
            print(f"Warning: Generator type {i} does not have 'maximum_level' key.")
        # Add constraint for minimum_level
        model.addConstraint(generator_output[(i, t)] >= generator_types[i]["minimum_level"] * generator_status[(i, t)])



In [20]:
# Solve the model
model.solve()

1

In [21]:
# Print the results
print("Generator Output:")
for i in range(len(generator_types)):
    for t in load_requirements.keys():
        print(f"Generator {i + 1}, Time {t}: {generator_output[i, t].value()} MW")


Generator Output:
Generator 1, Time 12 pm to 6 am: 17250.0 MW
Generator 1, Time 6 to 9 am: 34500.0 MW
Generator 1, Time 9 am to 3 pm: 28750.0 MW
Generator 1, Time 3 pm to 6 pm: 46000.0 MW
Generator 1, Time 6 pm to 12 pm: 31050.0 MW
Generator 2, Time 12 pm to 6 am: 0.0 MW
Generator 2, Time 6 to 9 am: 0.0 MW
Generator 2, Time 9 am to 3 pm: 0.0 MW
Generator 2, Time 3 pm to 6 pm: 0.0 MW
Generator 2, Time 6 pm to 12 pm: 0.0 MW


In [22]:
print("\nGenerator Status:")
for i in range(len(generator_types)):
    for t in load_requirements.keys():
        print(f"Generator {i + 1}, Time {t}: {'On' if generator_status[i, t].value() > 0 else 'Off'}")



Generator Status:
Generator 1, Time 12 pm to 6 am: Off
Generator 1, Time 6 to 9 am: Off
Generator 1, Time 9 am to 3 pm: Off
Generator 1, Time 3 pm to 6 pm: Off
Generator 1, Time 6 pm to 12 pm: Off
Generator 2, Time 12 pm to 6 am: Off
Generator 2, Time 6 to 9 am: Off
Generator 2, Time 9 am to 3 pm: Off
Generator 2, Time 3 pm to 6 pm: Off
Generator 2, Time 6 pm to 12 pm: Off


In [23]:
print("\nTotal Cost:", model.objective.value())



Total Cost: 786200.0


In [24]:
# Calculate the marginal cost of electricity production
marginal_cost = {}
for t in load_requirements.keys():
    marginal_cost[t] = 0
    for i in range(len(generator_types)):
        if generator_status[i, t].value() > 0 and generator_output[i, t].value() < generator_types[i]["maximum_level"]:
            marginal_cost[t] += generator_types[i]["cost_per_mw"]
print("\nMarginal Cost of Electricity Production:")
for t in load_requirements.keys():
    print(f"{t}: {marginal_cost[t]} $/MW")


Marginal Cost of Electricity Production:
12 pm to 6 am: 0 $/MW
6 to 9 am: 0 $/MW
9 am to 3 pm: 0 $/MW
3 pm to 6 pm: 0 $/MW
6 pm to 12 pm: 0 $/MW


In [25]:
# Calculate the cost of a 15% reduction in reserve output
cost_reduction = 0
for t in load_requirements.keys():
    cost_reduction += marginal_cost[t] * load_requirements[t] * 0.15
print("\nCost of a 15% Reduction in Reserve Output:", cost_reduction)


Cost of a 15% Reduction in Reserve Output: 0.0
